# RAG 증강, 생성
- Retrieval, Augmented,  Generation

# 환경변수 로딩

In [42]:
from dotenv import load_dotenv
import os


load_dotenv(override=True)

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']

# 필요한 라이브러리 로딩

In [43]:
from pinecone import Pinecone
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

# 검색

### 질문에 대한 top-k 검색

In [44]:
def sm_search_top_k(question):

    
    #벡터db, index 객체생성
    pc = Pinecone(api_key=PINECONE_API_KEY)
    
    index_name= "wiki"
    index = pc.Index(index_name)
    
    # embedding 객체 생성
    embedding = OpenAIEmbeddings(
        model = "text-embedding-3-small",
        # api_key = OPENAI_API_KEY # 생략가능
    )
    
    # #질문을 embedding vector로 변경 <----vector_store.similarity_search() 여기서 자동으로 됨
    # embedding_queestion = embedding.embed_documents(question)
    # embedding_queestion
    

    #top-k  검색
    # 텍스트 키는 "chunk_text"로 설정합니다.
    vector_store = PineconeVectorStore(index=index,        
                                       embedding=embedding, 
                                       text_key="full_text")  # default text 임, 우리가 설정한 값이 chunk_text
    
    # 질문에 대한 유사도 검색을 수행합니다.
    # query는 question 리스트의 첫 번째 요소를 사용하고, 상위 3개의 결과를 반환합니다.
    results = vector_store.similarity_search(
        query=question, # question 문자열을 embedding vector로 자동 변환됨
        k=3, 
        namespace="wiki_ns1")
    # print(results)
    
    # 검색된 문서의 메타데이터를 출력합니다.
    context= ""
    for result in results:
        # print(result)
        # print(result.metadata)
        # print(result.page_content)
        # print("-"*30)
        context = context + result.page_content
        
    return context

In [6]:
# 함수 호출
# question = "김대중 대통령의 행적에 대해 알려줘"
# top_k_context = sm_search_top_k(question)
# print(top_k_context)

취임식에는 김대중 전 대통령을 비롯해 김영삼, 전두환, 노태우, 최규하 전 대통령 등 전직 대통령 전원이 참석하였다. 대통령 비서실장으로 내정된 문희상 의원과 민정수석비서관 내정자였던 문재인도 참석했다. 김석수 국무총리, 박관용 국회의장, 최종영 대법원장, 유지담 중앙선거관리위원장, 고건 국무총리 내정자 등 5부 요인도 참석했다. 이 외에도 전국 광역단체장 전원이 취임식에 초청되었으며, 김종필 자민련 총재도 참석하여 삼김 전원이 참석한 마지막 대통령 취임식이 되었다. 문화계 인사로는 배우 문성근과 방송인 홍석천이 초청되었다. 외국 사절로는 콜린 파월 미 국무장관, 첸지천 중국 부총리, 고이즈미 준이치로 일본 총리, 나카소네 야스히로 전 일본 총리, 모리 요시로 전 일본 총리, 세르게이 미느로프 전 러시아1995년 부산 시장 선거 출마, 통합민주당 창당 활동에 합류
 1997년 새정치국민회의 입당, 새정치국민회의 부총재, 제15대 대통령선거 찬조 연설과 파랑새유세단 단장으로서의 활동 등으로 대통령 후보 김대중 지원
 1998년 서울 종로구 국회의원 보궐선거 당선, 현대자동차 파업 중재
 1999년 국회 동남아특위 위원장
 제6대 해양수산부 장관
 새천년민주당 당무위원
 새천년민주당 상임고문
 2002년 새천년민주당 제16대 대통령 후보
 2003년 ~ 2008년 제16대 대한민국 대통령아니라, 그동안 여러분에게 지고 있었던 마음의 짐을 풀었다는 것이 가장 기쁩니다"라는 말로 과거의 일을 반성했다. 그리고 그 해 12월 18일, 김대중이 15대 대통령에 당선되면서 노무현은 사상 처음으로 여당에 몸담게 되었다.


# 증강

## 프롬프트 템플릿 만들기

In [45]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# 프롬프트 템플릿
chat_templat = ChatPromptTemplate([
    # 시스템 프롬프트
    ("system", "당신은 역사전문가 특히 인문에 대해 상당한 조예가 깊습니다. 질문에 대해 최적의 답을 해주세요."),
     # user 프롬프트
    ("human", """ {question}에 대해 설명해주세요.
    아래의 context에서만 정보를 사용하고 다른 정보는 사용하지 마세요. 없으면 없다고 말해주세요.
                 context: {context}""")]
)
chat_templat

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 역사전문가 특히 인문에 대해 상당한 조예가 깊습니다. 질문에 대해 최적의 답을 해주세요.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template=' {question}에 대해 설명해주세요.\n    아래의 context에서만 정보를 사용하고 다른 정보는 사용하지 마세요. 없으면 없다고 말해주세요.\n                 context: {context}'), additional_kwargs={})])

## 질문 만들기

In [46]:
#질문
question = " 백남준의 예술 세계에 대해 설명해줘"
# question = " 유관순 열사의 희생 정신에 대해 설명해줘"
# question = "링컨 대통령에 대해 설명해주세요."


## 질문에 대한 top-k 검색

In [47]:
top_k_context = sm_search_top_k(question)
print(top_k_context)

관련 영상 
 백남준의 위성예술 1부 (1984년 방송)
 백남준의 위성예술 2부 (1984년 방송)
 백남준의 비디오아트세계 (1984년 방송)
 백남준, 예술로 미래를 말하다 (2006년 방송) - 백남준 추모특집

각주

외부 링크각주

외부 링크 

  백남준아트센터
 ARTCYCLOPEDIA에서 백남준 항목
 백남준의 세계 (출처:Guggenheim.org)
 백남준 간략한 일대기와 주요 작품 설명 (출처:Electronic Arts Intermix)
 백남준 일대기 (출처:@ MedienKunstNetz)}
 백남준 부고 "Father of Video Art Paik Nam-june Dies", (출처:조선일보, 2006년 1월 30일자 신문)
 백남준 부고 "Video artist Nam June Paik dead at 74", (출처:뉴욕 타임즈, 2006년 1월 30일 보도)
 백남준 공식 블로그세계’ 라는 대규모 회고전이 열렸으며, 이때 백남준은 레이저 아트 ‘야곱의 사다리’, ‘삼원소’ 등을 전시한 바 있다.


## 최종 프롬프트 만들기
- 템플릿 프롬프트 + 사용자질문 + top-k context

In [48]:
prompt = chat_templat.format_messages(
    question = question,
    context = top_k_context
)
prompt

[SystemMessage(content='당신은 역사전문가 특히 인문에 대해 상당한 조예가 깊습니다. 질문에 대해 최적의 답을 해주세요.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='  백남준의 예술 세계에 대해 설명해줘에 대해 설명해주세요.\n    아래의 context에서만 정보를 사용하고 다른 정보는 사용하지 마세요. 없으면 없다고 말해주세요.\n                 context: 관련 영상 \n 백남준의 위성예술 1부 (1984년 방송)\n 백남준의 위성예술 2부 (1984년 방송)\n 백남준의 비디오아트세계 (1984년 방송)\n 백남준, 예술로 미래를 말하다 (2006년 방송) - 백남준 추모특집\n\n각주\n\n외부 링크각주\n\n외부 링크 \n\n  백남준아트센터\n ARTCYCLOPEDIA에서 백남준 항목\n 백남준의 세계 (출처:Guggenheim.org)\n 백남준 간략한 일대기와 주요 작품 설명 (출처:Electronic Arts Intermix)\n 백남준 일대기 (출처:@ MedienKunstNetz)}\n 백남준 부고 "Father of Video Art Paik Nam-june Dies", (출처:조선일보, 2006년 1월 30일자 신문)\n 백남준 부고 "Video artist Nam June Paik dead at 74", (출처:뉴욕 타임즈, 2006년 1월 30일 보도)\n 백남준 공식 블로그세계’ 라는 대규모 회고전이 열렸으며, 이때 백남준은 레이저 아트 ‘야곱의 사다리’, ‘삼원소’ 등을 전시한 바 있다.', additional_kwargs={}, response_metadata={})]

## 생성
- LLM이 프롬프트에 따라 결과를 생성됨

In [49]:
# Openai gpt 모델 사용
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)

In [50]:
response = llm.invoke(prompt)

In [51]:
print(response.content)

백남준은 비디오 아트의 선구자로 알려져 있으며, 그의 예술 세계는 기술과 인간의 심리를 결합한 독창적인 작품들로 특징지어집니다. 방송된 영상들인 "백남준의 위성예술" 1부 및 2부, "백남준의 비디오아트세계", 그리고 "백남준, 예술로 미래를 말하다"는 그의 예술적 접근 방식과 철학을 다루고 있습니다.

특히 백남준은 새로운 매체인 비디오와 위성을 활용하여 예술의 경계를 넘었으며, 이는 그가 기술을 단순한 도구가 아닌 예술적 표현의 중요한 요소로 인식했음을 보여줍니다. 그의 작품들은 현대 사회의 변화와 미래를 반영하며, 기술과 인간의 관계에 대한 깊은 성찰을 제공합니다.

또한, 그는 "세계"라는 대규모 회고전에서도 레이저 아트 작품들을 전시했는데, 이는 그의 예술적 탐구가 계속해서 발전해왔음을 나타냅니다. 이러한 작품들은 비디오 아트의 가능성을 확장하며 예술과 기술의 융합을 탐구하는 중요한 사례로 여겨집니다. 

백남준의 예술 세계는 단순히 비디오라는 매체에 국한되지 않으며, 그의 작업은 시간, 공간, 인식의 새로운 방식을 제시합니다.


# LCEL chain으로 실행하기

In [52]:
# 실행 파이프라인 구성
# chain = 프롬프트 | llm | 출력파서(default str So 생략 가능)
chain = chat_templat | llm 

# 프롬프트 탬플릿의 변수에 대입할 값
input_data = {"question": question, "context":top_k_context}

# chain 실행
response = chain.invoke(input_data)

In [53]:
result = response.content

"백남준은 비디오 아트의 선구자로 알려져 있으며, 그의 예술 세계는 주로 미디어와 기술을 통한 커뮤니케이션의 변혁에 중점을 두고 있습니다. \n\n그의 대표적인 작업 중 일부는 위성을 활용한 예술을 포함합니다. '백남준의 위성예술' 시리즈(1984년 방송)에서는 미디어 아트와 글로벌 커뮤니케이션을 탐구하며 예술이 어떻게 국경을 넘어 소통할 수 있는지를 보여줍니다. 이러한 접근은 단순한 시청각적 경험을 넘어서, 정보와 문화의 교류를 시도한 것으로 해석될 수 있습니다.\n\n또한 '백남준의 비디오아트세계' (1984년 방송)에서는 그의 비디오 아트 작품이 가진 창의성과 혁신성을 강조합니다. 백남준은 비디오를 단순한 기록 매체가 아니라 새로운 표현 방식으로 바라보았으며, 이를 통해 관객과의 상호작용을 이끌어내는 예술을 창조했습니다.\n\n2006년 방송된 '백남준, 예술로 미래를 말하다'에서는 그의 예술적 비전과 철학이 어떻게 현대 사회와 미래에 대한 성찰을 포함하고 있는지를 설명해주고 있습니다. 백남준은 기술과 인문학적 접근을 결합하여, 예술이 미래 사회와 어떻게 연관될 수 있는지를 고민하였습니다.\n\n종합적으로, 백남준은 기술의 발전을 예술의 도구로 활용하면서도, 그 안에서 인간성과 소통의 가치를 잃지 않으려 했던 예술가의 모습이 두드러집니다. 이러한 요소들이 그의 예술 세계를 구성하고 있습니다."

In [55]:
result = "백남준은 비디오 아트의 선구자로 알려져 있으며, 그의 예술 세계는 주로 미디어와 기술을 통한 커뮤니케이션의 변혁에 중점을 두고 있습니다. \n\n그의 대표적인 작업 중 일부는 위성을 활용한 예술을 포함합니다. '백남준의 위성예술' 시리즈(1984년 방송)에서는 미디어 아트와 글로벌 커뮤니케이션을 탐구하며 예술이 어떻게 국경을 넘어 소통할 수 있는지를 보여줍니다. 이러한 접근은 단순한 시청각적 경험을 넘어서, 정보와 문화의 교류를 시도한 것으로 해석될 수 있습니다.\n\n또한 '백남준의 비디오아트세계' (1984년 방송)에서는 그의 비디오 아트 작품이 가진 창의성과 혁신성을 강조합니다. 백남준은 비디오를 단순한 기록 매체가 아니라 새로운 표현 방식으로 바라보았으며, 이를 통해 관객과의 상호작용을 이끌어내는 예술을 창조했습니다.\n\n2006년 방송된 '백남준, 예술로 미래를 말하다'에서는 그의 예술적 비전과 철학이 어떻게 현대 사회와 미래에 대한 성찰을 포함하고 있는지를 설명해주고 있습니다. 백남준은 기술과 인문학적 접근을 결합하여, 예술이 미래 사회와 어떻게 연관될 수 있는지를 고민하였습니다.\n\n종합적으로, 백남준은 기술의 발전을 예술의 도구로 활용하면서도, 그 안에서 인간성과 소통의 가치를 잃지 않으려 했던 예술가의 모습이 두드러집니다. 이러한 요소들이 그의 예술 세계를 구성하고 있습니다."

In [56]:
result

"백남준은 비디오 아트의 선구자로 알려져 있으며, 그의 예술 세계는 주로 미디어와 기술을 통한 커뮤니케이션의 변혁에 중점을 두고 있습니다. \n\n그의 대표적인 작업 중 일부는 위성을 활용한 예술을 포함합니다. '백남준의 위성예술' 시리즈(1984년 방송)에서는 미디어 아트와 글로벌 커뮤니케이션을 탐구하며 예술이 어떻게 국경을 넘어 소통할 수 있는지를 보여줍니다. 이러한 접근은 단순한 시청각적 경험을 넘어서, 정보와 문화의 교류를 시도한 것으로 해석될 수 있습니다.\n\n또한 '백남준의 비디오아트세계' (1984년 방송)에서는 그의 비디오 아트 작품이 가진 창의성과 혁신성을 강조합니다. 백남준은 비디오를 단순한 기록 매체가 아니라 새로운 표현 방식으로 바라보았으며, 이를 통해 관객과의 상호작용을 이끌어내는 예술을 창조했습니다.\n\n2006년 방송된 '백남준, 예술로 미래를 말하다'에서는 그의 예술적 비전과 철학이 어떻게 현대 사회와 미래에 대한 성찰을 포함하고 있는지를 설명해주고 있습니다. 백남준은 기술과 인문학적 접근을 결합하여, 예술이 미래 사회와 어떻게 연관될 수 있는지를 고민하였습니다.\n\n종합적으로, 백남준은 기술의 발전을 예술의 도구로 활용하면서도, 그 안에서 인간성과 소통의 가치를 잃지 않으려 했던 예술가의 모습이 두드러집니다. 이러한 요소들이 그의 예술 세계를 구성하고 있습니다."